# 영어 Word2Vec 만들기

- 영어로 된 코퍼스를 다운받아 전처리를 수행
- 전처리한 데이터를 바탕으로 Word2Vec 작업을 진행
- [영어/한국어 Word2Vec 실습](https://wikidocs.net/50739)

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

## 데이터 다운로드

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7fe10d553890>)

## 훈련 데이터 전처리하기

In [9]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [10]:
print('총 샘플의 개수 : {}'.format(len(result)))
for line in result[:3]:
    print(line)

총 샘플의 개수 : 273424
['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


## Word2Vec 훈련시키기

- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBOW, 1은 Skip-gram.

In [11]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

In [12]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8646872043609619), ('guy', 0.8039215207099915), ('lady', 0.7677041292190552), ('girl', 0.7556315064430237), ('boy', 0.7543885111808777), ('gentleman', 0.7195479869842529), ('soldier', 0.7041349411010742), ('kid', 0.6970995664596558), ('poet', 0.6769534945487976), ('surgeon', 0.6725988388061523)]


In [16]:
# woman = man - boy + girl
# topn: result length
model.wv.most_similar(positive=['man', 'girl'], negative=['boy'], topn=3)

[('woman', 0.8510866165161133),
 ('lady', 0.7480716109275818),
 ('guy', 0.7218708992004395)]

## Word2Vec 모델 저장하고 로드하기

In [13]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [14]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8646872043609619), ('guy', 0.8039215207099915), ('lady', 0.7677041292190552), ('girl', 0.7556315064430237), ('boy', 0.7543885111808777), ('gentleman', 0.7195479869842529), ('soldier', 0.7041349411010742), ('kid', 0.6970995664596558), ('poet', 0.6769534945487976), ('surgeon', 0.6725988388061523)]


In [17]:
!ls -l

total 97816
-rw-r--r-- 1 root root 25622357 Feb 24 01:03 eng_w2v
drwxr-xr-x 1 root root     4096 Feb 22 14:38 sample_data
-rw-r--r-- 1 root root 74533638 Feb 24 00:58 ted_en-20160408.xml
